In [3]:
import pandas as pd
import io, gzip
from codecs import decode
import pysrt
import datetime as dt
import base64
import re
import dill
import numpy as np
import sys
from xmlrpc.client import ServerProxy
import os as os
import os.path
import json
import xarray as xr

In [9]:
with open('credentials.pkd','rb') as fp:
    credentials = dill.load(fp)
USER_AGENT=credentials['user']
USERNAME=credentials['user']
PASS=credentials['passw']

In [156]:
OPENSUBTITLES_SERVER = 'http://api.opensubtitles.org/xml-rpc'
LANGUAGE = 'en'

xmlrpc = ServerProxy(OPENSUBTITLES_SERVER,allow_none=True)
data = xmlrpc.LogIn(USERNAME, PASS, LANGUAGE, USER_AGENT)
status=data.get('status')
if status != '200 OK':
    print('Could not log in. Something is wrong... ')
    print("data.get('status')")
    token = None
else:
    token = data.get('token')
    print('Logged in. Acquired token = {}'.format(token))


Logged in. Acquired token = g1oSyN1VQpb,G,nNHKyGPImKaxd


In [157]:
data

{'data': {'Content-Location': 'https://vip-api.opensubtitles.org/xml-rpc',
  'DownloadCnt': '2479',
  'IDUser': '5937493',
  'IsVIP': 1,
  'UploadCnt': '0',
  'UserNickName': 'matrixx105',
  'UserPreferedLanguages': 'eng',
  'UserRank': 'vip member',
  'UserWebLanguage': 'en'},
 'seconds': 0.027,
 'status': '200 OK',
 'token': 'g1oSyN1VQpb,G,nNHKyGPImKaxd'}

In [155]:
dir(xmlrpc)

['_ServerProxy__allow_none',
 '_ServerProxy__close',
 '_ServerProxy__encoding',
 '_ServerProxy__handler',
 '_ServerProxy__host',
 '_ServerProxy__request',
 '_ServerProxy__transport',
 '_ServerProxy__verbose',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [131]:
xmlrpc.NoOperation(token)

{'download_limits': {'client_24h_download_count': 2,
  'client_24h_download_limit': 1000,
  'client_download_quota': 998,
  'client_ip': '71.183.36.65',
  'global_24h_download_limit': 200,
  'limit_check_by': 'user_id',
  'user_id': '5937493'},
 'seconds': 0.002,
 'status': '200 OK'}

In [133]:
xmlrpc.ServerInfo(token)

{'application': 'OpenSuber v0.2',
 'contact': 'admin@opensubtitles.org',
 'download_limits': {'client_24h_download_count': 2,
  'client_24h_download_limit': 1000,
  'client_download_quota': 998,
  'client_ip': '71.183.36.65',
  'global_24h_download_limit': 200,
  'limit_check_by': 'user_id',
  'user_id': '5937493'},
 'last_update_strings': {'ar': '2007-02-03 21:36:14',
  'at': '2016-02-28 01:41:54',
  'bg': '2017-01-30 15:37:22',
  'br': '2016-02-25 05:52:22',
  'ca': '2014-11-13 11:32:44',
  'cs': '2014-12-25 01:00:17',
  'da': '2015-06-16 14:25:51',
  'de': '2014-11-13 05:01:32',
  'el': '2015-09-07 11:58:19',
  'en': '2014-11-12 12:34:41',
  'eo': '2014-11-13 18:27:49',
  'es': '2014-11-20 13:11:05',
  'et': '2014-11-12 16:31:04',
  'eu': '2014-11-13 17:08:41',
  'fa': '2007-02-03 21:36:16',
  'fi': '2014-11-12 13:16:05',
  'fr': '2014-11-13 08:02:00',
  'gl': '2007-10-11 16:23:40',
  'he': '2007-02-03 21:36:17',
  'hi': '2016-09-08 15:51:39',
  'hr': '2014-11-13 01:08:44',
  'hu': 

In [28]:
with open('movie_DB_1_7_2018.pkd', 'rb') as fp:
    movie_database = dill.load(fp)

In [63]:
tid=movie_database[4444]['imdb_id']

In [64]:
tid.strip('tt')

'5052448'

In [107]:
td=search_imdbid(tid.strip('tt'))

In [128]:
sr=[(item['SubDownloadsCnt'],item['SubRating'],item['IDSubtitleFile'],item['MovieReleaseName']) for item in td 
 if item['SubFormat']=='srt' if not re.findall('.*[^a-zA-Z]cd[^a-zA-Z]',item['MovieReleaseName'].lower())]

In [130]:
sr_sort=sorted(sr, key=lambda x: -int(x[0]))
sr_sort[0:6]

[('121014', '10.0', '1951976249', '720p.HDDVD.DTS.x264-ESiR'),
 ('100383', '0.0', '1952936725', 'The.Matrix.1999.720p.BrRip.x264.YIFY'),
 ('60255', '0.0', '1953233027', 'The.Matrix.1999.1080p.BluRay.x264-CtrlHD'),
 ('58548', '10.0', '1954137017', 'The.Matrix.1999.720p.BrRip.x264.YIFY'),
 ('52782', '10.0', '1951661350', 'The.Matrix.1999.720p.HDTV.x264-THOR'),
 ('24315', '7.0', '1952184164', 'BluRay.1080p.x264.DTS-WiKi')]

In [103]:
td[2]

{'IDMovie': '505336',
 'IDMovieImdb': '5052448',
 'IDSubMovieFile': '0',
 'IDSubtitle': '6978149',
 'IDSubtitleFile': '1955558448',
 'ISO639': 'en',
 'LanguageName': 'English',
 'MatchedBy': 'imdbid',
 'MovieByteSize': '0',
 'MovieFPS': '23.976',
 'MovieHash': '0',
 'MovieImdbRating': '7.7',
 'MovieKind': 'movie',
 'MovieName': 'Get Out',
 'MovieNameEng': '',
 'MovieReleaseName': 'Get.Out.2017.720p.BluRay.x264-SPARKS',
 'MovieTimeMS': '0',
 'MovieYear': '2017',
 'QueryCached': 1,
 'QueryNumber': '0',
 'QueryParameters': {'imdbid': '5052448', 'sublanguageid': 'eng'},
 'Score': 34.84414,
 'SeriesEpisode': '0',
 'SeriesIMDBParent': '0',
 'SeriesSeason': '0',
 'SubActualCD': '1',
 'SubAddDate': '2017-05-11 19:40:03',
 'SubAuthorComment': '',
 'SubAutoTranslation': '0',
 'SubBad': '0',
 'SubComments': '0',
 'SubDownloadLink': 'http://dl.opensubtitles.org/en/download/src-api/vrf-19dc0c5f/sid-3h98pjKhp3BGfBM5Y-JQCLtz7od/file/1955558448.gz',
 'SubDownloadsCnt': '124414',
 'SubEncoding': 'UTF-8

In [144]:
for i in range(44):
    search_data=xmlrpc.SearchSubtitles(token,[{'sublanguageid': 'eng','imdbid': '5052448'}])
    print(search_data['status'])

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


In [153]:
search_data['status']

'200 OK'

In [41]:
def search_imdbid(s_imdb):
    #s_imdb='1055369'
    search_data=xmlrpc.SearchSubtitles(token,[{'sublanguageid': 'eng','imdbid': s_imdb}])
    data=search_data['data']
    return data

def find_best_file(data,rank):
    best=[]
    for i in range(0,len(data)):
        if data[i]['SubFormat'] == 'srt' and 'cd' not in data[i]['SubFileName'].lower():
            #print(data[i]['SubFileName'].lower(),data[i]['IDSubtitleFile'],float(data[i]['SubRating'])*float(data[i]['SubDownloadsCnt']))
            #best.append(float(data[i]['SubRating'])*float(data[i]['SubDownloadsCnt'])) 
            best.append(float(data[i]['SubDownloadsCnt']))    
        else:
            best.append(0)
    
    if best:
        best_sub_id=data[best.index(max(best))]['IDSubtitleFile'] 
        best[best.index(max(best))]=0
        sbest_sub_id=data[best.index(max(best))]['IDSubtitleFile'] 
        best[best.index(max(best))]=0
        tbest_sub_id=data[best.index(max(best))]['IDSubtitleFile'] 
        #print('Found the best one', best_sub_id)
    if rank==1 and best:
        return best_sub_id
    elif rank==2 and best:
        return sbest_sub_id
    elif rank==3 and best:
        return tbest_sub_id
    else:
        return False

def download_sub(best_sub_id):
    if best_sub_id:
        #best_sub_id='1952195191'
        down_data=xmlrpc.DownloadSubtitles(token, [best_sub_id])
        if down_data['status'] !='200 OK':
            print('--------------------- Error Message:')
            print(down_data['status'])
            exit("Exiting Program Now. Come back Tomorrow. ")
        return down_data
    else:
        return False
    
def decode_parse(down_data,imdb_id):
    if down_data:
        compressed_subdata = decode(bytearray(down_data['data'][0]['data'].encode("utf-8")),'base64')
        sub_text = gzip.GzipFile(fileobj=io.BytesIO(compressed_subdata)).read()
        #parse it and store it into a DataFrame with three columns, starttime, endtime, and line
        sub_text_str = str(sub_text)
        snew = sub_text_str.replace("\\n","\n").replace("\\r","\r").replace("</i>","").replace("<i>-","").replace("<i>","").replace("\\","")
        subs=list(filter(None, snew.split('\r\n')))
        subs=[x for x in subs if not x.isdigit()]
        if subs[0][0]=='b':
            subs.pop(0)

        starttimes=[]
        endtimes=[]
        words=[]
        #"-->" in line
        prevLineWasWords =False
        for line in subs:
            line=line.strip()
            if "-->" in line:
                starttimes.append(line[:8])
                endtimes.append(line[17:25])
            elif len(line) > 1:
                    if prevLineWasWords:
                            words[len(words)-1] = words[len(words)-1] + " " + line
                    else:
                            words.append(line)
                    prevLineWasWords = True
                    continue
            prevLineWasWords = False  

        #this needs to be made better but for now we need these to be same length so it will have to do
        trim_by=min([len(starttimes),len(endtimes),len(words)])
        starttimes=starttimes[0:trim_by]
        endtimes=endtimes[0:trim_by]
        words=words[0:trim_by]

        imdb_idi=[imdb_id]
        linfo=['StartTime','EndTime','Line']
        index=pd.MultiIndex.from_product([imdb_idi,linfo],names=['IDs','LData'])
        plines=pd.DataFrame(columns=index)


        s1 = pd.Series(starttimes)
        s2 = pd.Series(endtimes)
        s3 = pd.Series(words)
        plines.loc[:,(imdb_id,'StartTime')] = s1.values
        plines.loc[:,(imdb_id,'EndTime')] = s2.values
        plines.loc[:,(imdb_id,'Line')] = s3.values
        return plines
    else:
        return False

#def populate_db(list):
    #input is a list of IMDB_IDS
    #open existing database of subtitles
    #check if each id is in there
        #if not download it and add it to the list (do this 20 at a time)
        #if yes then move on

In [16]:
##open existing database and download shit from it
if os.path.isfile('movbaseB.h5'):
    movdb=pd.read_hdf('movbaseB.h5', 'imdb_id')
else:
    print('Cannot find file movbaseB!')

In [10]:
batch=movdb[1000:1350]
len(batch)

350

In [11]:
#run this PER GENRE
#input is a list of imdb_id's which can come from whatever
#probably should come from a subset of movdb
if os.path.isfile('Sub_data.h5'):
    Sub_data=pd.read_hdf('Sub_data.h5', 'subs')
else:
    print('Cannot find file Sub_data! Making new one.')

for item in batch['imdb_id']:
    if item not in Sub_data.columns.levels[0]:
            one_sub=decode_parse(download_sub(find_best_file(search_imdbid(item),1)),item)
            try:
                if not one_sub:
                    print('COULDNT FIND SUBS FOR {}.   Title:  {} '.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))
                    continue
            except:
                if len(one_sub) < 2:
                    one_sub=decode_parse(download_sub(find_best_file(search_imdbid(item),2)),item)

                    if len(one_sub) < 2:
                        one_sub=decode_parse(download_sub(find_best_file(search_imdbid(item),3)),item)

                        if len(one_sub) <2:
                            print('First 3 subs for {} were bad. Not going any further. Title:  {} '.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))
                            continue



                Sub_data=pd.concat([Sub_data,one_sub], axis=1)
                Sub_data=Sub_data.fillna('')
                Sub_data.to_hdf('Sub_data.h5', 'subs', mode='w')
                print('Downloaded subs for {}.   Title:  {} '.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))

    else:
        print('Encountered double of {}. Title:  {}'.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))
        continue
        
print('Done. Writing Sub_data to file.')

Downloaded subs for 3387542.   Title:  The Forest 
Downloaded subs for 0130018.   Title:  I Still Know What You Did Last Summer 
Downloaded subs for 1311067.   Title:  Halloween II 
Downloaded subs for 0433386.   Title:  The Grudge 2 
Downloaded subs for 0099528.   Title:  The Exorcist III 
Downloaded subs for 0097368.   Title:  The Fly II 
Downloaded subs for 0450345.   Title:  The Wicker Man 
Downloaded subs for 0091419.   Title:  Little Shop of Horrors 
Downloaded subs for 1053859.   Title:  The Grudge 3 
Downloaded subs for 0192731.   Title:  Urban Legends: Final Cut 
Downloaded subs for 0220506.   Title:  Halloween: Resurrection 
Downloaded subs for 0475944.   Title:  The Covenant 
Downloaded subs for 0105217.   Title:  Raising Cain 
Downloaded subs for 0155975.   Title:  Psycho 
Downloaded subs for 1270761.   Title:  Don't Be Afraid of the Dark 
Downloaded subs for 0083972.   Title:  Friday the 13th Part III 
Downloaded subs for 0096734.   Title:  The 'Burbs 
Downloaded subs for 

COULDNT FIND SUBS FOR .   Title:  The Story Of Short Stack 
Downloaded subs for 0098213.   Title:  Roger & Me 
Downloaded subs for 1664894.   Title:  Cave of Forgotten Dreams 
Downloaded subs for 0226648.   Title:  Wolves 
Downloaded subs for 0088178.   Title:  Stop Making Sense 
Downloaded subs for 1587707.   Title:  Exit Through the Gift Shop 
Downloaded subs for 0379225.   Title:  The Corporation 
Downloaded subs for 1529567.   Title:  Sea Rex 3D: Journey to a Prehistoric World 
COULDNT FIND SUBS FOR 4266692.   Title:  Drain the Bermuda Triangle 
Downloaded subs for 0100332.   Title:  Paris is Burning 
Downloaded subs for 0388789.   Title:  Born into Brothels 
COULDNT FIND SUBS FOR 3635920.   Title:  D-Day: Normandy 1944 
Downloaded subs for 0109508.   Title:  Crumb 
Downloaded subs for 0389326.   Title:  Riding Giants 
Downloaded subs for 3882074.   Title:  The Eagle Huntress 
Downloaded subs for 0342172.   Title:  Capturing the Friedmans 
Downloaded subs for 1584016.   Title:  Cat

Downloaded subs for 1322313.   Title:  Sunshine Superman 
Downloaded subs for 1112115.   Title:  King Corn 
COULDNT FIND SUBS FOR 0216731.   Title:  Edgeplay: A Film About The Runaways 
Downloaded subs for 1990352.   Title:  The Island President 
Downloaded subs for 0970521.   Title:  Turtle: The Incredible Journey 
COULDNT FIND SUBS FOR 1541666.   Title:  Circo 
COULDNT FIND SUBS FOR 1746153.   Title:  Crime After Crime 
COULDNT FIND SUBS FOR 0430916.   Title:  Beautiful Losers 
COULDNT FIND SUBS FOR .   Title:  The Story Of Short Stack 
COULDNT FIND SUBS FOR 2007995.   Title:  Screaming in High Heels: The Rise & Fall of th.. 
Downloaded subs for 3268458.   Title:  The Internet's Own Boy: The Story of Aaron Swartz 
COULDNT FIND SUBS FOR 0264802.   Title:  Live Nude Girls Unite! 
Downloaded subs for 3270538.   Title:  Requiem for the American Dream 
COULDNT FIND SUBS FOR 1276943.   Title:  Bringing Godzilla Down to Size 
COULDNT FIND SUBS FOR 2234315.   Title:  Occupy Unmasked 
COULDNT

In [12]:
read_data=pd.read_hdf('Sub_data.h5', 'subs')
len(read_data.columns.levels[0])

1601